In [1]:
%matplotlib inline

http://pytorch.org/tutorials/beginner/blitz/autograd_tutorial.html#sphx-glr-beginner-blitz-autograd-tutorial-py
Autograd: automatic differentiation
===================================

<u>Central to all neural networks in PyTorch is the ``autograd`` package.</u>
Let’s first briefly visit this, and we will then go to training our
first neural network.


<u>The ``autograd`` package provides automatic differentiation for all operations on Tensors.</u>
It is a define-by-run framework, which means that your backprop is defined by how your code is run, and that every single iteration can be different.

Let us see this in more simple terms with some examples.

Variable
---------

``autograd.Variable`` is the central class of the package.
It wraps a Tensor, and supports nearly all of operations defined on it.
Once you finish your computation you can call ``.backward()`` and have all the gradients computed automatically.

You can access the raw tensor through the ``.data`` attribute, while the gradient w.r.t. this variable is accumulated into ``.grad``.


![Variable](Variable.png)

   Variable

There’s one more class which is very important for autograd implementation - a ``Function``.

<u>``Variable`` and ``Function`` are interconnected and build up an acyclic graph, that encodes a complete history of computation.</u>
<u>Each variable has a ``.grad_fn`` attribute that references a ``Function`` that has created the ``Variable`` (except for Variables created by the user - their ``grad_fn is None``).</u>

If you want to compute the derivatives, you can call ``.backward()`` on a ``Variable``.
If ``Variable`` is a scalar (i.e. it holds a one element data), you don’t need to specify any arguments to ``backward()``, however if it has more elements, you need to specify a ``grad_output`` argument that is a tensor of matching shape.

In [2]:
import torch

'''
http://pytorch.org/docs/master/autograd.html

Automatic differentiation package - torch.autograd
torch.autograd provides classes and functions implementing automatic differentiation of arbitrary scalar
valued functions.
It requires minimal changes to the existing code - you only need to wrap all tensors in Variable objects.

http://pytorch.org/docs/master/autograd.html#variable

Variable API is nearly the same as regular Tensor API
(with the exception of a couple in-place methods, that would overwrite inputs required for gradient computation).
Wraps a tensor and records the operations applied to it.
'''
from torch.autograd import Variable

Create a variable:



In [3]:
x = Variable(torch.ones(2, 2), requires_grad=True) # http://pytorch.org/docs/master/_modules/torch/autograd/variable.html#Variable
print(x)

Variable containing:
 1  1
 1  1
[torch.FloatTensor of size 2x2]



In [4]:
print("data: ", x.data) #Wrapped tensor of any type
print("grad: ", x.grad) #Variable holding the gradient of type and location matching the .data
print("requires_grad", x.requires_grad) #Boolean indicating whether the Variable has been created by a subgraph containing any Variable, that requires it
print("volatile", x.volatile) #Boolean indicating that the Variable should be used in inference mode, i.e. don’t save the history.
print("is_leaf", x.is_leaf) #Boolean indicating if the Variable is a graph leaf (i.e if it was created by the user)
print("grad_fn", x.grad_fn) #Gradient function graph trace

('data: ', 
 1  1
 1  1
[torch.FloatTensor of size 2x2]
)
('grad: ', None)
('requires_grad', True)
('volatile', False)
('is_leaf', True)
('grad_fn', None)


Do an operation of variable:



In [5]:
y = x + 2
print(y)

Variable containing:
 3  3
 3  3
[torch.FloatTensor of size 2x2]



``y`` was created as a result of an operation, so it has a ``grad_fn``.



In [6]:
print(y.grad_fn) #grad_fn – Gradient function graph trac

Do more operations on y



In [7]:
z = y * y * 3
out = z.mean() # http://pytorch.org/docs/master/torch.html#torch.mean

print(z, out)

(Variable containing:
 27  27
 27  27
[torch.FloatTensor of size 2x2]
, Variable containing:
 27
[torch.FloatTensor of size 1]
)


Gradients
---------
let's backprop now
``out.backward()`` is equivalent to doing ``out.backward(torch.Tensor([1.0]))``

http://pytorch.org/docs/master/autograd.html#torch.autograd.Variable.backward <br>
backward(gradient=None, retain_graph=None, create_graph=None, retain_variables=None)<br>
Computes the gradient of current variable w.r.t. graph leaves<br>
The graph is differentiated using the <b>chain rule</b>.
<u>If the variable is non-scalar (i.e. its data has more than one element) and requires gradient, the function additionally requires specifying gradient</u>

In [8]:
out.backward() # can be executed only once

print gradients d(out)/dx




In [9]:
print(x.grad) # grad – Variable holding the gradient of type and location matching the .data

Variable containing:
 4.5000  4.5000
 4.5000  4.5000
[torch.FloatTensor of size 2x2]



You should have got a matrix of ``4.5``. Let’s call the ``out``
*Variable* “$o$”.
We have that <br>
$o = \frac{1}{4}\sum_i z_i$, <br>
$z_i = 3(x_i+2)^2$ and <br>
$z_i\bigr\rvert_{x_i=1} = 27$.<br>
Therefore,<br>
$\frac{\partial o}{\partial x_i} = \frac{3}{2}(x_i+2)$, hence<br>
$\frac{\partial o}{\partial x_i}\bigr\rvert_{x_i=1} = \frac{9}{2} = 4.5$.



You can do many crazy things with autograd!



In [16]:
x = torch.randn(3)
print(x)
x = Variable(x, requires_grad=True)

y = x * 2

# the difference (norm) between two numbers < 1000
# http://mathonline.wikidot.com/the-norm-of-a-vector#toc4
c = 0
while y.data.norm() < 1000:
    y = y * 2
    c += 1

print(y)
print(c)


 0.0546
 2.3461
-0.1120
[torch.FloatTensor of size 3]

Variable containing:
   27.9359
 1201.1937
  -57.3287
[torch.FloatTensor of size 3]

8


In [17]:
gradients = torch.FloatTensor([0.1, 1.0, 0.0001])
print(gradients)
y.backward(gradients)

print(x.grad)


 1.0000e-01
 1.0000e+00
 1.0000e-04
[torch.FloatTensor of size 3]

Variable containing:
  51.2000
 512.0000
   0.0512
[torch.FloatTensor of size 3]



**Read Later:**

Documentation of ``Variable`` and ``Function`` is at
http://pytorch.org/docs/autograd

